In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon
import plotly.express as px


In [2]:
# Attempt to refactor plot function:

# def create_plot(plot_func, args):
#     fig = plot_func(args[0]
#                         , lat='LATITUDE'
#                         , lon='LONGITUDE')
#                         , center=dict(lat=40.71, lon=-74)
#                         , zoom=8.9
#                         , mapbox_style="open-street-map"
#                         , *args[1:])
#     return fig

In [16]:
collisions = gpd.read_file("data/nypd-motor-vehicle-collisions_head_cut.csv")

In [21]:
for i in collisions.latitude:
        i.astype(float)
        print(i)

AttributeError: 'str' object has no attribute 'astype'

In [17]:
# Remove rows with unknown Lat or Long
collisions = collisions[collisions.LATITUDE !=""]
collisions = collisions[collisions.LONGITUDE!=""]
collisions.rename(columns={'LATITUDE':'latitude','LONGITUDE':'longitude'}, inplace=True)

collisions.shape

## More general way of doing this:
# collisions.replace('',float("NaN"),inplace=True)
# collisions.dropna(subset=['LONGITUDE','LATITUDE'],inplace=True)

(910, 3)

In [7]:
collisions.geometry = gpd.points_from_xy(collisions.longitude.astype(float), collisions.latitude.astype(float))
collisions.crs = {'init': 'epsg:4326'}    # Plain lat/long is 4326
collisions = collisions.to_crs(epsg=2263) # Move to the CRS of the hosital data (geometry column changes, lat/long don't)
collisions.head()

ValueError: could not convert string to float: 

In [ ]:
fig = px.density_mapbox(collisions
                        , lat='latitude'
                        , lon='longitude'
                        , radius=8
                        , center=dict(lat=40.71, lon=-74)
                        , zoom=8.9
                        , mapbox_style="open-street-map"
                        #, range_color=(0,1) 
                       )
fig.show()

In [ ]:
hospitals = gpd.read_file("data/hospitals/hospitals.shp")
hospitals.head()

In [ ]:
hospitals.crs

In [ ]:
# Fill null capacities with median capacity
hospitals = hospitals.fillna(hospitals.capacity.median())
hospitals.capacity = hospitals.capacity.astype(float)
hospitals.name     = hospitals.name.str.title()

In [ ]:
fig2 = px.scatter_mapbox(hospitals
                         , lat='latitude'
                         , lon='longitude'
                         , size='capacity'
                         , center=dict(lat=40.7, lon=-73.9)
                         , zoom=9
                         , color = 'capacity'
                         , mapbox_style="open-street-map"
                         , color_continuous_scale='Bluered_r'
                         , opacity = .9
                        )
fig2.show()

In [ ]:
fig2 = px.scatter_mapbox(hospitals
                         , lat='latitude'
                         , lon='longitude'
                         , size='capacity'
                         , text='name'
                         , center=dict(lat=40.7, lon=-73.9)
                         , zoom=9
                         , mapbox_style="open-street-map"
                         , opacity = .9
                        )
fig2.show()

In [ ]:
# Create a 10km buffer around each hospital
coverage = pd.DataFrame(hospitals.geometry.buffer(10000), columns = ['buffer'])
near_hospitals = hospitals.geometry.buffer(10000).unary_union
near_hospitals

In [ ]:
# Which collisions are outside of this hospital catchment area?
outside_range = collisions[~collisions.geometry.apply(lambda x: near_hospitals.contains(x))]
outside_range.shape


In [ ]:
# Plot the uncovered collisions along with the hospital locations
fig3 = px.density_mapbox(outside_range
                         , lat='latitude'
                         , lon='longitude'
                         , radius=8
                         , center=dict(lat=40.71, lon=-74)
                         , zoom=8.9
                         , mapbox_style="open-street-map"
                         , range_color=(0,5))

fig3.add_trace(fig2.data[0])
fig3.show()